In [1]:
%reset
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.spatial import ConvexHull

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.DoubleTensor)

In [2]:
data = pickle.load(open("../../data/basipetal.pkl", 'rb'))

Dx = 0.
Dy = 0.
height_source = 38.
height_target = 100.

source = torch.tensor(data['source_d'], dtype=torch.get_default_dtype())
target = torch.tensor(data['target_d'], dtype=torch.get_default_dtype())
source_curve = torch.tensor(data['source_c'], dtype=torch.get_default_dtype())
target_curve = torch.tensor(data['target_c'], dtype=torch.get_default_dtype())

smin, smax = torch.min(source_curve[:, 1]), torch.max(source_curve[:, 1])
sscale = height_source / (smax - smin)
source[:, 1] = Dy - sscale * (source[:, 1] - smax)
source[:, 0] = Dx + sscale * (source[:, 0] - torch.mean(source_curve[:, 0]))
source_curve[:, 1] = Dy - sscale * (source_curve[:, 1] - smax)
source_curve[:, 0] = Dx + sscale * (source_curve[:, 0] - torch.mean(source_curve[:, 0]))

tmin, tmax = torch.min(target_curve[:, 1]), torch.max(target_curve[:, 1])
tscale = height_target / (tmax - tmin)
target[:, 1] = - tscale * (target[:, 1] - tmax)
target[:, 0] = tscale * (target[:, 0] - torch.mean(target_curve[:, 0]))
target_curve[:, 1] = - tscale * (target_curve[:, 1] - tmax)
target_curve[:, 0] = tscale * (target_curve[:, 0] - torch.mean(target_curve[:, 0]))

source_curve_fit = source_curve.clone()
source_curve_fit = source_curve_fit[source_curve_fit[:, 0] <= 0]
target_curve_fit = target_curve.clone()
target_curve_fit = target_curve_fit[target_curve_fit[:, 0] <= 0]


hull_fit = ConvexHull(source_curve)
source_curve_convex = source_curve[hull_fit.vertices]

aabb_source = dm.Utilities.AABB.build_from_points(source_curve)

In [3]:
# pts_implicit1_step = 1.5
# pts_implicit1_x, pts_implicit1_y = torch.meshgrid([
#     torch.arange(0., aabb_source.xmin-pts_implicit1_step, step=-pts_implicit1_step),
#     torch.arange(aabb_source.ymin, aabb_source.ymax+pts_implicit1_step, step=pts_implicit1_step)])

# pts_implicit1 = dm.Utilities.grid2vec(pts_implicit1_x, pts_implicit1_y)
# pts_implicit1_mask = dm.Utilities.is_inside_shape(source_curve_convex, pts_implicit1)
# pts_implicit1 = pts_implicit1[pts_implicit1_mask]

length = 12
mesh_pts_x, mesh_pts_y = torch.meshgrid([torch.linspace(0, 1, length), torch.linspace(0, 1, length)])
pts_implicit1 = dm.Utilities.grid2vec(mesh_pts_x, mesh_pts_y)
source = pts_implicit1.clone()

In [4]:
# A polynomial model of order 3
def pol(pos, a, b, c, d, e, f, g, h, i, j):
    return a \
        + b*pos[:, 0] + c*pos[:, 1] \
        + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1] \
        + g*pos[:, 0]**3 + h*pos[:, 1]**3 + i*pos[:, 0]**2*pos[:, 1] + j*pos[:, 0]*pos[:, 1]**2

In [5]:
sigma_gt = 0.5

# We create a target
torch.manual_seed(1337)
silent_curve_target = dm.DeformationModules.SilentLandmarks.build_from_points(source_curve)
#silent_curve_target.manifold.fill_cotan(2.*torch.rand_like(source_curve).view(-1).requires_grad_())
silent_dots_target = dm.DeformationModules.SilentLandmarks.build_from_points(source)
#silent_dots_target.manifold.fill_cotan(2.*torch.rand_like(source).view(-1).requires_grad_())

# The model parameters we use and would like to get back
abc_gt = torch.tensor([[1., 0.],
                       [4., 0.], [1., 1.],
                       [0., 1.], [0.5, 2.], [1., 1.],
                       [0., 0.1], [0.1, -0.2], [0., 0.], [0., 0.1]])

th = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R_target = torch.stack([dm.Utilities.rot2d(t) for t in th])
C_target = pol(pts_implicit1, abc_gt[0].unsqueeze(1), abc_gt[1].unsqueeze(1), abc_gt[2].unsqueeze(1), abc_gt[3].unsqueeze(1), abc_gt[4].unsqueeze(1), abc_gt[5].unsqueeze(1), abc_gt[6].unsqueeze(1), abc_gt[7].unsqueeze(1), abc_gt[8].unsqueeze(1), abc_gt[9].unsqueeze(1)).transpose(0, 1).unsqueeze(2)

imp1_pts_cotan = 10.*torch.rand_like(pts_implicit1)
imp1_R_cotan = 10.*torch.rand_like(R_target)

implicit1_target = dm.DeformationModules.ImplicitModule1(dm.Manifolds.Stiefel(2, pts_implicit1.shape[0], gd=(pts_implicit1.view(-1).requires_grad_(), R_target.view(-1).requires_grad_()), cotan=(imp1_pts_cotan.clone().view(-1).requires_grad_(), imp1_R_cotan.clone().view(-1).requires_grad_())), C_target, sigma_gt, 0.01, 1.)

compound_target = dm.DeformationModules.CompoundModule([silent_curve_target, silent_dots_target, implicit1_target])
dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian(compound_target), 20, "midpoint")

target_curve = silent_curve_target.manifold.gd.detach().view(-1, 2)
target = silent_dots_target.manifold.gd.detach().view(-1, 2)
pts_implicit1_target = implicit1_target.manifold.gd[0].detach().view(-1, 2)

aabb_target = dm.Utilities.AABB.build_from_points(target_curve)

In [7]:
%matplotlib qt5

plt.axis('equal')
plt.plot(target.numpy()[:, 0], target.numpy()[:, 1], '.')
plt.plot(source.numpy()[:, 0], source.numpy()[:, 1], 'x')
plt.show()

In [8]:
def myParametricModel(modules, fitted_parameters):
    abc = fitted_parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    g = abc[6].unsqueeze(1)
    h = abc[7].unsqueeze(1)
    i = abc[8].unsqueeze(1)
    j = abc[9].unsqueeze(1)
    modules[1]._ImplicitModule1__C = pol(pts_implicit1, a, b, c, d, e, f, g, h, i, j).transpose(0, 1).unsqueeze(2)

In [35]:
torch.manual_seed(1234)
#sigma = sigma_gt
sigma = 1.
nu = 0.01
coeff = 1.
abc = torch.zeros(10, 2)
abc[0, 0] = 1.
abc[0, 1] = 1.
# abc = abc_gt.clone() + 0.001*torch.randn_like(abc)
# print(abc)
# print(abc_gt)
noise_sigma = 0.
C = pol(pts_implicit1,
        abc[0].unsqueeze(1),
        abc[1].unsqueeze(1), abc[2].unsqueeze(1),
        abc[3].unsqueeze(1), abc[4].unsqueeze(1), abc[5].unsqueeze(1),
        abc[6].unsqueeze(1), abc[7].unsqueeze(1), abc[8].unsqueeze(1), abc[9].unsqueeze(1)).t().unsqueeze(2)
abc.requires_grad_()
th = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(t) for t in th])


In [36]:
abcs = [abc.detach().clone()]
def log_abc(model):
    abcs.append(model.parameters[-1].detach().clone())

implicit1 = dm.DeformationModules.ImplicitModule1(
    dm.Manifolds.Stiefel(2, pts_implicit1.shape[0],
                         gd=(pts_implicit1.view(-1).requires_grad_(),
                             R.view(-1).requires_grad_())),
    C, sigma, nu, coeff)

#implicit0 = dm.DeformationModules.ImplicitModule0.build_from_points(2, pts_implicit1.shape[0], 0.15, 0.01, coeff=100, gd=pts_implicit1.view(-1).requires_grad_())

model = dm.Models.ModelPointsRegistration(
    [source],
    [implicit1],
    [dm.Attachment.EuclideanPointwiseDistanceAttachement(10.)],
    other_parameters=[abc],
    precompute_callback=myParametricModel)

fitter = dm.Models.ModelFittingScipy(model, 1., 1000., post_iteration_callback=log_abc)

costs = []

In [ ]:
costs.extend(fitter.fit([target], 250, log_interval=1))

In [11]:
plt.plot(range(len(costs)), costs)
plt.show()

In [ ]:
# Gradcheck
torch.set_printoptions(precision=15)
gradcheck1 = False
gradcheck2 = False
if gradcheck1:
    print("Gradcheck for the first pass (moment fitting)...")
    implicit = dm.DeformationModules.ImplicitModule1(
        dm.Manifolds.Stiefel(2, pts_implicit1.shape[0],
                             gd=(pts_implicit1.view(-1).requires_grad_(),
                                 R.view(-1).requires_grad_()),
                             cotan=(0.1*torch.ones_like(pts_implicit1).view(-1).requires_grad_(),
                                    0.1*torch.ones_like(R).view(-1).requires_grad_())),
        C, sigma1, nu1, coeff1)

    model = dm.Models.ModelPointsRegistration(
        [source],
        [implicit],
        [dm.Attachment.EuclideanPointwiseDistanceAttachement()])

    print(model.gradcheck([target], 1e-2))

if gradcheck2:
    print("Gradcheck for the second pass (polynome fitting)...")
    implicit = dm.DeformationModules.ImplicitModule1(
        dm.Manifolds.Stiefel(2, pts_implicit1.shape[0],
                    gd=(pts_implicit1.view(-1).requires_grad_(),
                        R.view(-1).requires_grad_()),
                    cotan=(0.1*torch.ones(pts_implicit1.shape).view(-1).requires_grad_(),
                           0.1*torch.ones(R.shape).view(-1).requires_grad_())),
        C, sigma1, nu1, coeff1)

    model = dm.Models.ModelPointsRegistration(
        [source],
        [implicit],
        [dm.Attachment.EuclideanPointwiseDistanceAttachement()],
        other_parameters=[abc],
        precompute_callback=myParametricModel,
        fit_moments=False)

    print(model.gradcheck([target], 10.))

In [15]:
azer = np.zeros([20, len(abcs)])
for i in range(len(abcs)):
    for j in range(20):
        azer[j, i] = abcs[i].view(-1)[j].item()

In [17]:
for i in range(20):
    plt.subplot(5, 5, i+1)
    plt.plot(range(len(abcs)), azer[i])

plt.show()

In [38]:
a_ob = abcs[-1][0].detach().unsqueeze(1)
b_ob = abcs[-1][1].detach().unsqueeze(1)
c_ob = abcs[-1][2].detach().unsqueeze(1)
d_ob = abcs[-1][3].detach().unsqueeze(1)
e_ob = abcs[-1][4].detach().unsqueeze(1)
f_ob = abcs[-1][5].detach().unsqueeze(1)
g_ob = abcs[-1][6].detach().unsqueeze(1)
h_ob = abcs[-1][7].detach().unsqueeze(1)
i_ob = abcs[-1][8].detach().unsqueeze(1)
j_ob = abcs[-1][9].detach().unsqueeze(1)
obtained_C = pol(pts_implicit1, a_ob, b_ob, c_ob, d_ob, e_ob, f_ob, g_ob, h_ob, i_ob, j_ob).transpose(0, 1).unsqueeze(2)

In [23]:
print(torch.cat([C, obtained_C, C_target], dim=2))

tensor([[[ 1.0000e+00, -5.2003e-02,  1.0000e+00],
         [ 1.0000e+00, -9.2507e-02,  0.0000e+00]],

        [[ 1.0000e+00,  6.0925e-02,  1.0951e+00],
         [ 1.0000e+00, -5.0778e-02,  1.0729e-01]],

        [[ 1.0000e+00,  1.6283e-01,  1.1989e+00],
         [ 1.0000e+00,  3.7425e-03,  2.4673e-01]],

        [[ 1.0000e+00,  2.6104e-01,  1.3119e+00],
         [ 1.0000e+00,  6.9596e-02,  4.1743e-01]],

        [[ 1.0000e+00,  3.6288e-01,  1.4346e+00],
         [ 1.0000e+00,  1.4533e-01,  6.1848e-01]],

        [[ 1.0000e+00,  4.7569e-01,  1.5672e+00],
         [ 1.0000e+00,  2.2947e-01,  8.4899e-01]],

        [[ 1.0000e+00,  6.0680e-01,  1.7104e+00],
         [ 1.0000e+00,  3.2058e-01,  1.1080e+00]],

        [[ 1.0000e+00,  7.6353e-01,  1.8646e+00],
         [ 1.0000e+00,  4.1718e-01,  1.3947e+00]],

        [[ 1.0000e+00,  9.5322e-01,  2.0302e+00],
         [ 1.0000e+00,  5.1783e-01,  1.7082e+00]],

        [[ 1.0000e+00,  1.1832e+00,  2.2077e+00],
         [ 1.0000e+00,  6.2106e-

In [39]:
%matplotlib qt5
from matplotlib.patches import Ellipse
compound = dm.DeformationModules.CompoundModule(model.modules)
compound.manifold.fill(model.init_manifold)
inter = dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian(compound), 20, 'midpoint')

init = model.init_manifold.copy()

init_dots = init[0].gd.detach().view(-1, 2)
#init_curve = init[1].gd.detach().view(-1, 2)
dots = compound[0].manifold.gd.detach().view(-1, 2)
#curve = compound[1].manifold.gd.detach().view(-1, 2)

plt.axis('equal')
plt.plot(init_dots[:, 0].numpy(), init_dots[:, 1].numpy(), '.k')
#plt.plot(init_curve[:, 0].numpy(), init_curve[:, 1].numpy(), color='black')

plt.plot(dots[:, 0].numpy(), dots[:, 1].numpy(), '.r')
#plt.plot(curve[:, 0].numpy(), curve[:, 1].numpy(), '--r')

plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), 'xb')
#plt.plot(target_curve_fit[:, 0].numpy(), target_curve_fit[:, 1].numpy(), 'b')
plt.show()

In [41]:

plt.axis('equal')
scale1 = 0.01
scale2 = 0.025
ax1 = plt.subplot(1, 2, 1)
for i in range(pts_implicit1.shape[0]):
    C_i = scale1*C_target[i, :, 0]
    ell = Ellipse(xy=pts_implicit1[i], width=abs(C_i[0]), height=abs(C_i[1]), angle=0.)
    ax1.add_artist(ell)

ax2 = plt.subplot(1, 2, 2)
for i in range(pts_implicit1.shape[0]):
    C_i = scale2*obtained_C[i, :, 0]
    ell = Ellipse(xy=pts_implicit1[i], width=abs(C_i[0]), height=abs(C_i[1]), angle=0.)
    ax2.add_artist(ell)
    

In [40]:
print(torch.stack([model.parameters[-1].detach().view(-1), abc_gt.view(-1)]).transpose(0, 1))
print(abc_gt)

tensor([[ 0.4780,  1.0000],
        [ 0.4357,  0.0000],
        [ 4.1044,  4.0000],
        [ 3.2192,  0.0000],
        [-2.4235,  1.0000],
        [-5.7991,  1.0000],
        [-0.7574,  0.0000],
        [-4.9407,  1.0000],
        [ 7.0960,  0.5000],
        [12.7040,  2.0000],
        [-2.6425,  1.0000],
        [ 1.6101,  1.0000],
        [-1.7149,  0.0000],
        [ 1.9464,  0.1000],
        [-3.0245,  0.1000],
        [-5.4161, -0.2000],
        [ 5.7877,  0.0000],
        [ 3.3547,  0.0000],
        [-2.9058,  0.0000],
        [-5.4792,  0.1000]])
tensor([[ 1.0000,  0.0000],
        [ 4.0000,  0.0000],
        [ 1.0000,  1.0000],
        [ 0.0000,  1.0000],
        [ 0.5000,  2.0000],
        [ 1.0000,  1.0000],
        [ 0.0000,  0.1000],
        [ 0.1000, -0.2000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.1000]])


In [55]:
my_abc = []

In [78]:
my_abc.append(model.parameters[-1].detach().clone())

In [79]:
print(my_abc)

[tensor([[  6.6762,   0.0397],
        [ 36.4753,  -4.3668],
        [ 10.5294,   6.3414],
        [-14.7929,  23.4392],
        [ -1.7233,  20.5477],
        [ -5.3204,   1.5438],
        [  8.3466, -11.1785],
        [  5.9666,  -6.4049],
        [ 18.9232,   3.5250],
        [ -5.8314,   3.1615]]), tensor([[  5.6487,  -0.1454],
        [ 13.1704,   6.3433],
        [  4.7976,   5.1645],
        [ 22.5308, -11.8791],
        [  3.1212,  18.8180],
        [  3.7168, -14.8044],
        [-15.9482,  13.4765],
        [ -2.5173, -10.1246],
        [-14.4891,   8.9616],
        [ 17.3202,  12.7672]]), tensor([[  6.0789,   0.9451],
        [ 17.1798,  -2.9912],
        [  5.6734,  -2.6579],
        [ 14.7377,  21.5298],
        [ -4.8407,  37.5969],
        [  8.8885, -21.9956],
        [-10.7783, -11.9094],
        [  6.8898, -19.4336],
        [ -5.9481,  21.5884],
        [  4.1380,   7.2174]])]
